In [298]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os
import re
import itertools
import datetime
import time

Populating the interactive namespace from numpy and matplotlib


In [299]:
hdfsclient = Client("amp-spark-master.amp", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [300]:
executor_mem = "100g"
cores_per_executor = 30
num_executors = 14

In [301]:
val_results = []

In [304]:
num_feats = [32768]
regs = [1e10]
seq_sizes = [1000]
kmer_sizes = [8]
weights = [0.0]
gammas = [1.0]
solver_sample = [1.0]
params = itertools.product(num_feats, regs, seq_sizes, gammas, kmer_sizes, weights, solver_sample)

In [ ]:
# this is where the magic happen
for num_feat, reg, seq_size, gamma, kmer, weight, solver_sample in params:
        params = {"num_feats":num_feats, "gamma":gamma, "reg":reg, "seq_size":seq_size, "kmer_size": kmer, "weight": weight, "solver_sample": solver_sample}
        print("COMPUTING EXPERIMENT FOR {0}".format(params))
        val_df = run_deepsea_pipeline(['ATF3'], hdfs=hdfsclient, seq_size=seq_size,weight=weight, run_solver_only=True, gamma=gamma, reg=reg, kmer_size=kmer, solve_sample=solver_sample, num_filters=num_feat, num_filters_to_use=32768)
        val_result = compute_results(val_df, "val_", params, control_rocs=control_rocs, control_prs=control_prs)
        val_results.append(val_result)
        pd.concat(val_results).to_csv("/home/eecs/vaishaal/deepsea_exps/{0}_val_results".format(time.time()))
        print(val_result)
    
    

COMPUTING EXPERIMENT FOR {'num_feats': [32768], 'weight': 0.0, 'solver_sample': 1.0, 'seq_size': 1000, 'kmer_size': 8, 'reg': 10000000000.0, 'gamma': 1.0}
WEIGHT 0.0
['/user/vaishaal/.Trash', '/user/vaishaal/.sparkStaging', '/user/vaishaal/.staging', '/user/vaishaal/1200_8_8192_0.1_train.features', '/user/vaishaal/1200_8_8192_0.1_val.features', '/user/vaishaal/1200_8_8192_1.0E-5_train.features', '/user/vaishaal/1200_8_8192_1.0E-5_val.features', '/user/vaishaal/1200_8_8192_1.0_train.features', '/user/vaishaal/1200_8_8192_1.0_val.features', '/user/vaishaal/1200_8_8192_10.0_train.features', '/user/vaishaal/1200_8_8192_10.0_val.features', '/user/vaishaal/1200_8_8192_100.0_train.features', '/user/vaishaal/1200_8_8192_100.0_val.features', '/user/vaishaal/4000_8_32768_1.0_train.features', '/user/vaishaal/4000_8_32768_1.0_val.features', '/user/vaishaal/400_8_8192_0.1_train.features', '/user/vaishaal/400_8_8192_0.1_val.features', '/user/vaishaal/400_8_8192_1.0E-5_train.features', '/user/vaishaa

In [110]:
val_results_df = pd.concat(val_results)

In [111]:
val_results_df["kmer_size"] = val_results_df["kmer_size"].fillna(8)

In [ ]:
parts

In [160]:
val_df_filtered = val_results_df
sns.stripplot(x="num_feats", y="val_pr_auc", hue="seq_size", data=val_df_filtered, jitter=True)
plt.legend(bbox_to_anchor=(1.5, 0.7))

TypeError: unhashable type: 'list'

In [55]:
sns.stripplot(y="val_pr_auc", x="reg", hue="meta", data=pd.concat(val_results))
plt.legend(bbox_to_anchor=(1.5, 0.7))

ValueError: No objects to concatenate

In [137]:
print(pd.concat(val_results).to_csv())

,control_pr_auc,control_roc_auc,gamma,meta,num_feats,reg,seq_size,val_pr_auc,val_roc_auc
0,0.496891,0.973567,1.0,k562|atf3|none,[1024],1e-08,200,0.0423686017413,0.889271982615
1,0.368892,0.993995,1.0,hepg2|atf3|none,[1024],1e-08,200,0.0984265507085,0.738658160287
2,0.0869624,0.995998,1.0,h1-hesc|atf3|none,[1024],1e-08,200,0.00722428159496,0.968265382691
3,0.280633,0.934871,1.0,a549|atf3|etoh_0.02pct,[1024],1e-08,200,0.0647103837107,0.707543037975
4,0.0034965,0.964491,1.0,gm12878|atf3|none,[1024],1e-08,200,0.000449277885476,0.740497624406
0,0.496891,0.973567,1.0,k562|atf3|none,[1024],1e-08,200,0.0423686017413,0.889271982615
1,0.368892,0.993995,1.0,hepg2|atf3|none,[1024],1e-08,200,0.0984265507085,0.738658160287
2,0.0869624,0.995998,1.0,h1-hesc|atf3|none,[1024],1e-08,200,0.00722428159496,0.968265382691
3,0.280633,0.934871,1.0,a549|atf3|etoh_0.02pct,[1024],1e-08,200,0.0647103837107,0.707543037975
4,0.0034965,0.964491,1.0,gm12878|atf3|none,[1024],1e-08,200,0.000449277885476,0.740497624406
0

In [14]:
def compute_results(df, tag, params={}, control_rocs={}, control_prs={}):
    metas = list(set(map(lambda x:  re.sub(r"(_label|_score).*", "", x), df.columns.values)))
    results = []
    for m in metas:
        result = {} 
        score_column = m + "_score"
        label_column = m + "_label"
        y_pred = df[score_column]
        y_true = df[label_column]
        print("NUM TRUE {0}".format(sum(y_true)))
        print("NUM TOTAL {0}".format(len(y_true)))
        roc_auc = metrics.roc_auc_score(y_true, y_pred)
        pr_auc = metrics.average_precision_score(y_true, y_pred)
        result[tag + 'roc_auc'] = roc_auc
        result[tag + 'pr_auc'] = pr_auc
        if (control_rocs != {}):
            result["control_roc_auc"] = control_rocs[m.lower()]
        if (control_prs != {}):
            result["control_pr_auc"] = control_prs[m.lower()]
            
        result['meta'] = m.lower()
        result = dict(result.items() + params.items())
        results.append(result)
    return pd.DataFrame(results)

In [13]:
res = compute_results(val_df, "val_", control_rocs=control_rocs, control_prs=control_prs)

NUM TRUE 24.0
NUM TOTAL 8000
NUM TRUE 6.0
NUM TOTAL 8000
NUM TRUE 4.0
NUM TOTAL 8000
NUM TRUE 100.0
NUM TOTAL 8000
NUM TRUE 2.0
NUM TOTAL 8000


In [59]:
res[['meta','val_roc_auc','control_roc_auc', 'val_pr_auc', 'control_pr_auc']]

NameError: name 'res' is not defined

In [61]:
res[cols]

NameError: name 'res' is not defined

In [62]:
res.plot(kind="scatter", y="val_roc_auc", x="control_roc_auc")
plt.plot( [0,1],[0,1] )
plt.xlim(0,1.0)
plt.ylim(0,1.0)


NameError: name 'res' is not defined

In [11]:
control_df = pd.read_csv("/home/eecs/vaishaal/deepsea_data/deepsea_control_results_eval.csv")

In [12]:
control_rocs = {}
control_prs = {}
for c in control_df.itertuples():
   meta_key = "{0}|{1}|{2}".format(c[1], c[2], c[3]).lower() 
   control_rocs[meta_key] = c[4]
   control_prs[meta_key] = c[5]

In [71]:
val_df.to_csv()

A549|ATF3|EtOH_0.02pct_label  GM12878|ATF3|None_label  \
0 0 0 0 0 0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
...                                   ...                      ...   
          0                             0                        0   
          0                             1                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0                             0                        0   
          0            

In [109]:
pd.read_csv('/tmp/deepsea_val_results')

,,,,,,,,,,A549|ATF3|EtOH_0.02pct_label,GM12878|ATF3|None_label,H1-hESC|ATF3|None_label,HepG2|ATF3|None_label,K562|ATF3|None_label,K562|ATF3|None_label.1,A549|ATF3|EtOH_0.02pct_score,GM12878|ATF3|None_score,H1-hESC|ATF3|None_score,HepG2|ATF3|None_score,K562|ATF3|None_score
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [123]:
len(open("/tmp/deepsea_val_results").readlines()[0].split(","))

12

In [125]:
len(open("/tmp/deepsea_val_results").readlines()[1].split(","))

18

In [186]:
datetime.datetime.now()

datetime.datetime(2017, 2, 10, 14, 28, 41, 782003)

1486765755.44609